## Testing Automatic Differentiation of Derivative Operators:
#### Gradient, Jacobian, Divergence, Laplacian, Curl

In [1]:
import torch
import DRLPDE_nn

In [2]:
### Initial Variables
num_points = 5
dim_in = 3
dim_out = 2

x = torch.randn(num_points, dim_in).requires_grad_(True)
y = torch.empty(num_points, dim_out)

In [3]:
### [y0, y1] = F(x0, x1, t)

y[:,0] = torch.cos(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])
y[:,1] = -torch.sin(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])
p = (torch.cos(2*x[:,0]) + torch.cos(2*x[:,1]) )*torch.exp(-4*x[:,2])/4

In [4]:
### Check Jacobian
J = DRLPDE_nn.jacobian(y,x)
dy = torch.empty(num_points, dim_out, dim_in)

dy[:,0,0] = -torch.sin(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,0,1] =  torch.cos(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,0,2] =  -2*torch.cos(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])

dy[:,1,0] = -torch.cos(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,1,1] = torch.sin(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,1,2] = 2*torch.sin(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])

In [5]:
A = DRLPDE_nn.advection(y,x,J)
Ay = torch.empty(num_points, dim_out)

Ay[:,0] = - torch.cos(x[:,0]) * torch.sin(x[:,0]) * torch.exp(-4*x[:,2])
Ay[:,1] = - torch.cos(x[:,1]) * torch.sin(x[:,1]) * torch.exp(-4*x[:,2])

In [6]:
L = DRLPDE_nn.laplace(y,x,J)
Ly = torch.empty(num_points, dim_out)

Ly[:,0] = -2*torch.cos(x[:,0])*torch.sin(x[:,1])*torch.exp(-2*x[:,2])
Ly[:,1] = 2*torch.sin(x[:,0])*torch.cos(x[:,1])*torch.exp(-2*x[:,2])


In [7]:
L - Ly

tensor([[5.9605e-08, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.1921e-07],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]], grad_fn=<SubBackward0>)

In [71]:
lap = torch.empty(x.size(0), y.size(1))
for ii in range(y.size(1)):
    grad = J[:,ii,:y.size(1)]
    lap[:,ii] = DRLPDE_nn.divergence(grad,x)

In [40]:
div = torch.empty(x.size(0))
for ii in range(y.size(1)):
    div += DRLPDE_nn.gradient(y[:,ii],x)[:,ii]

tensor([[ 1.0412e+01, -1.8279e+01],
        [-1.3303e-01, -9.5216e-03],
        [ 7.6546e+00, -4.1802e+00],
        [ 4.8353e-01, -6.7343e-01],
        [-4.2685e+00,  6.6426e+00]], grad_fn=<CopySlices>)

In [72]:
J[:,0,:]

tensor([[ 3.5476e+00,  1.3413e+01,  1.0412e+01],
        [ 2.9693e-02, -1.0664e-02, -1.3303e-01],
        [ 9.0713e-01,  8.8183e+00,  7.6546e+00],
        [ 2.6333e-01,  3.0914e-01,  4.8353e-01],
        [ 2.0782e+00,  3.4110e+00, -4.2685e+00]], grad_fn=<SliceBackward0>)

In [41]:
time_deriv + A - L

tensor([[-1.2023e+02, -1.0813e+30],
        [-2.6403e-01,  1.9894e-02],
        [-2.0666e+33, -1.0813e+30],
        [ 7.9930e-01,  1.5103e+00],
        [ 7.2271e+00, -2.7467e+01]], grad_fn=<SubBackward0>)

In [43]:
DRLPDE_nn.gradient(p,x)[:,:2]

tensor([[-1.4106e+02,  1.0225e+02],
        [ 2.0258e-03,  8.5067e-04],
        [-2.1903e+01,  3.5646e+01],
        [-1.6776e-01,  1.6341e-01],
        [ 1.5764e+01, -1.4182e+01]], grad_fn=<SliceBackward0>)

In [24]:
J[0,:,:]

tensor([[  3.5476,  13.4128,  10.4124],
        [-13.4128,  -3.5476,  18.2793]], grad_fn=<SliceBackward0>)

In [25]:
dy = torch.empty(num_points, dim_out, dim_in)

dy[:,0,0] = -torch.sin(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,0,1] =  torch.cos(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,0,2] =  -2*torch.cos(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])

dy[:,1,0] = torch.cos(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,1,1] = -torch.sin(x[:,0]) * torch.sin(x[:,1]) * torch.exp(-2*x[:,2])
dy[:,1,2] = 2*torch.sin(x[:,0]) * torch.cos(x[:,1]) * torch.exp(-2*x[:,2])

In [26]:
DRLPDE_nn.gradient(y[:,0], x)

tensor([[ 3.5476e+00,  1.3413e+01,  1.0412e+01],
        [ 2.9693e-02, -1.0664e-02, -1.3303e-01],
        [ 9.0713e-01,  8.8183e+00,  7.6546e+00],
        [ 2.6333e-01,  3.0914e-01,  4.8353e-01],
        [ 2.0782e+00,  3.4110e+00, -4.2685e+00]], grad_fn=<AddBackward0>)

In [27]:
dy[:,0,:]

tensor([[ 3.5476e+00,  1.3413e+01,  1.0412e+01],
        [ 2.9693e-02, -1.0664e-02, -1.3303e-01],
        [ 9.0713e-01,  8.8183e+00,  7.6546e+00],
        [ 2.6333e-01,  3.0914e-01,  4.8353e-01],
        [ 2.0782e+00,  3.4110e+00, -4.2685e+00]], grad_fn=<SliceBackward0>)

10

torch.Size([10, 3, 4])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])